In [1]:


import os
import json
import csv
from symtable import Symbol
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pprint
import talib
import btalib
from datetime import datetime


In [2]:


from time import sleep
from binance.client import Client
from binance.streams import ThreadedWebsocketManager



In [7]:
def get_data_frames():
    startTime = "3 month ago UTC"

    # valid intervals - 1m, 3m, 5m, 15m, 30m, 1h, 2h, 4h, 6h, 8h, 12h, 1d, 3d, 1w, 1M
    interval = "1h"

    bars = client.futures_historical_klines(symbol, interval, startTime)

    for line in bars:
        del line[5:]

    df = pd.DataFrame(bars, columns=['date', 'open', 'high', 'low', 'close'])
    return df

In [8]:
def bb_trade_logic():
    data = get_data_frames()
    #set RSI tp
    rsi_period = 14
    

    # close_list = pd.to_numeric(data['close'], downcast='float')
    # open_list = pd.to_numeric(data['open'], downcast='float')
    # high_list = pd.to_numeric(data['high'], downcast='float')
    # low_list = pd.to_numeric(data['low'], downcast='float')

    #Get RSI
    data['rsi_14']=talib.RSI(data['close'],5)
    #Get timeframe
    data['date'] = pd.to_datetime(data['date'], unit='ms')

    #QQE Logic
    data['rsii']= talib.EMA(data['rsi_14'],5)
    data['tr']=0.0
    data['truerange']=0.0
    data['wwma']=0.0
    data['atrsi']=0.0
    data['QQES']=0.0
    data['QDN']=0.0
    data['QUP']=0.0
    data['ATR']=0.0
    data['QQEF']=talib.EMA(data['rsi_14'],5)
    tt=1/(14.0)



    for index, row in data.iterrows():
        if(index>17):
            
            data.at[index,'tr']=abs(data.at[index,'rsii']-data.at[index-1,'rsii'])
            
            a1=abs(float(data.at[index,'high'])-float((data.at[index,'low'])))
            b1=abs(float(data.at[index,'high'])-float(data.at[index-1,'close']))
            c1=abs(float(data.at[index,'low'])-float(data.at[index-1,'low']))

            if(a1>b1 and a1>c1):
                data.at[index,"truerange"]=a1
            if(a1<b1 and b1>c1):
                data.at[index,"truerange"]=b1
            if(c1>a1 and c1>b1):
                data.at[index,"truerange"]=c1
                
            for x in range(index-15, index):
                data.at[index,'ATR']+=data.at[x,"truerange"]/14.0
            
            data.at[index,'wwma']=tt*(data.at[index,'tr'])+(1-tt)*(data.at[index-1,'wwma'])
        
            data.at[index,'atrsi']= tt*(data.at[index,'wwma'])+(1-tt)*(data.at[index-1,'atrsi'])
            
            data.at[index,'QUP']=data.at[index,'QQEF']+data.at[index,'atrsi']*4.236
            
            data.at[index,'QDN']=data.at[index,'QQEF']-data.at[index,'atrsi']*4.236            
            
            if(data.at[index,'QUP']<data.at[index-1,'QQES']):
                data.at[index,'QQES']=data.at[index,'QUP']
            else:
                if(data.at[index,'QQEF']>data.at[index-1,'QQES'] and data.at[index-1,'QQEF']<data.at[index-1,'QQES']):
                    data.at[index,'QQES']=data.at[index,'QDN']
                else:
                    if(data.at[index,'QDN']>data.at[index-1,'QQES']):
                        data.at[index,'QQES']=data.at[index,'QDN']
                    else:
                        if(data.at[index,'QQEF']<data.at[index-1,'QQES']  and data.at[index-1,'QQEF']>data.at[index-1,'QQES'] ):
                            data.at[index,'QQES']=data.at[index,'QUP']
                        else:
                            data.at[index,'QQES']=data.at[index-1,'QQES'] 
            

            data.at[index,'sell'] = np.where((data.at[index-1,'QQEF'] < data.at[index-1,'QQES'] and data.at[index,'QQEF'] > data.at[index,'QQES']), data.at[index,'close'],np.NAN)
            data.at[index,'sellatr']=np.where((data.at[index-1,'QQEF'] < data.at[index-1,'QQES'] and data.at[index,'QQEF'] > data.at[index,'QQES']), data.at[index,'ATR'],np.NAN)
            data.at[index,'buy'] = np.where((data.at[index-1,'QQES'] < data.at[index-1,'QQEF'] and data.at[index,'QQES'] > data.at[index,'QQEF']),data.at[index-1,'close'],np.NAN)
            data.at[index,'buyatr']=np.where((data.at[index-1,'QQES'] < data.at[index-1,'QQEF'] and data.at[index,'QQES'] > data.at[index,'QQEF']),data.at[index,'ATR'],np.NAN)

    buy_atr=data['buyatr'].tolist() 
    sell_atr=data['sellatr'].tolist()    
    buy_price = data['buy'].tolist()
    sell_price = data['sell'].tolist()
    open_price=data['open'].tolist()
    close_price=data['close'].tolist()

    with open('output.txt', 'w') as f:
        f.write(data.to_string())
    calculateProfit(buy_price, sell_price,buy_atr,sell_atr,open_price,close_price)


In [12]:
def calculateProfit(buy_price, sell_price,buy_atr,sell_atr,open_price,close_price):
    investment = 10000
    brokerage = 0.00016
    profit = 0
    diff=0.0
    tick1=0
    latestprice=-1.0
    latestatr=-1.0
    currentorder=0
    for index in range(len(buy_price)):
        if(tick1==1 and currentorder!=0):
            # We can only sell now
            print("Cover long position")
            if(float(lastestprice-latestatr) > float(open_price[index]) or float(lastestprice+2.0*latestatr) < float(open_price[index])):
                profit += ((investment)/lastestprice)*(float(open_price[index])-lastestprice)
                investment+= profit
                latestatr=-1.0
                latestprice=-1.0
                currentorder=0
            
        if(tick1==-1 and currentorder!=0):
            # We can only buy now
            print("Cover short position")
            if(float(lastestprice-2.0*latestatr) > float(open_price[index]) or float(lastestprice+latestatr) < float(open_price[index])):
                profit += ((investment)/lastestprice)*(lastestprice-float(open_price[index]))
                investment+=profit
                latestatr=-1.0
                latestprice=-1.0
                currentorder=0


        if(tick1 == 0 and currentorder == 0):
            # Check for possible trades
            if(float(float(buy_price[index]))>float(0)):
                tick1=1
                lastestprice=float(buy_price[index])
                latestatr=float(buy_atr[index])
                currentorder+=1
                # Use buy function
                # buy_order = client.order_market_buy(symbol='some crypto', quantity='buy quantity in number')

            if(float(sell_price[index])>float(0)):
                tick1=-1
                lastestprice=float(close_price[index])
                latestatr=float(sell_atr[index])
                currentorder+1
                # Use sell funtion
                # buy_order = client.order_market_sell(symbol='some crypto', quantity='buy quantity in number')

    print("profit-",profit)
    print("investment-", investment)

In [13]:


def main():
    bb_trade_logic()


if __name__ == "__main__":
    # api_key = os.environ.get('binance_api')
    # api_secret = os.environ.get('binance_secret')

    api_key = 'Okzore7olB305rjB9ufDHDGa8z3oBeOdqUECgFX9Dg4MCHLPnQUKN5N6FE6hL89h'
    api_secret = 'OEMLJZYS1awB8eMyBJKmSXVCUKwvrKtvnRASLzBHX0OfmHROq64tkbNuFZPvK3Sr'

    client = Client(api_key, api_secret)

    # pprint.pprint(client.get_account())
    # pprint.pprint(client.futures_account_balance())

    symbol = 'MATICUSDT'

    main()



Cover long position
profit- -61.29499609940878
investment- 9938.705003900592


In [11]:


def buy_or_sell(buy_price, sell_price,buy_atr,sell_atr,open_price,close_price):

    # print('price: ', current_price)
    # if buy_sell_list[-1] == 1.0:
    #     if current_price < df['buy'][-1]:
    #         print('buy buy buy.......')
            # buy_order = client.order_market_buy(symbol='some crypto', quantity='buy quantity in number')
    #         # print(buy_order)
    # elif buy_sell_list[-1] == -1.0:
    #     if current_price > df['sell'][-1]:
    #         print('sell sell sell.......')
            # buy_order = client.order_market_sell(symbol='some crypto', quantity='buy quantity in number')
    #         # print(buy_order)

    # else:
    #     print('noting to do....')

    lastestprice = -1.0
    latestatr = -1.0
    tick1 = 0
    order = False
    for index in range(len((open_price))):    
        if((float(lastestprice))>float(0)):
            if(float(lastestprice-latestatr) > float(open_price[index]) or float(lastestprice+2.0*latestatr) < float(open_price[index])):
                # buy_order = client.order_market_sell(symbol='some crypto', quantity='buy quantity in number')
                # print(buy_order)
                lastestprice = -1.0
                latestatr = -1.0
                tick1 = 0
                order = False

            
            if(float(lastestprice-2.0*latestatr) > float(open_price[index]) or float(lastestprice+latestatr) < float(open_price[index])):
                # buy_order = client.order_market_buy(symbol='some crypto', quantity='buy quantity in number')
                # print(buy_order
                lastestprice = -1.0
                latestatr = -1.0
                tick1 = 0
                order = False
            


        if(float(float(buy_price[index]))>float(0)):
            tick1=1
            lastestprice=float(buy_price[index])
            latestatr=float(buy_atr[index])
        if(float(sell_price[index])>float(0)):
            tick1=-1
            lastestprice=float(close_price[index])
            latestatr=float(sell_atr[index])
        if(latestprice>0 and order!=False):
            # Buy ATR logic
            if(tick1 == 1):
                # buy_order = client.order_market_buy(symbol='some crypto', quantity='buy quantity in number')
                # print(buy_order)
                order=True
                    

            # Sell ATR logic
            if(tick1 == -1):
                # buy_order = client.order_market_sell(symbol='some crypto', quantity='buy quantity in number')
                # print(buy_order)
                order=True

